In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    "book-crossings/BX-Books.csv",
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

print(type(df_books))
print(df_books.head())

df_ratings = pd.read_csv(
    "book-crossings/BX-Book-Ratings.csv",
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

print(type(df_ratings))
print(df_ratings.head())

In [ ]:
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

active_users = user_counts[user_counts >= 200].index
popular_books = book_counts[book_counts >= 100].index

df_ratings = df_ratings[df_ratings['user'].isin(active_users)]
df_ratings = df_ratings[df_ratings['isbn'].isin(popular_books)]
df_books = df_books[df_books['isbn'].isin(df_ratings['isbn'].unique())]


user_book_matrix = df_ratings.pivot_table(index='isbn', columns='user', values='rating', fill_value=0)
matrix_sparse = csr_matrix(user_book_matrix.values)

isbn_to_title = dict(zip(df_books['isbn'], df_books['title']))
title_to_isbn = dict(zip(df_books['title'], df_books['isbn']))

knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(matrix_sparse)

In [ ]:
def get_recommends(book=""):
    if book not in title_to_isbn:
        return [book, []]
    isbn = title_to_isbn[book]
    if isbn not in user_book_matrix.index:
        return [book, []]
    idx = user_book_matrix.index.get_loc(isbn)
    distances, indices = knn_model.kneighbors(matrix_sparse[idx], n_neighbors=6)
    recs = []
    for i, d in zip(indices[0][1:], distances[0][1:]):  # skip itself
        rec_isbn = user_book_matrix.index[i]
        rec_title = isbn_to_title.get(rec_isbn, "Unknown")
        recs.append([rec_title, float(d)])
    return [book, recs]

In [ ]:
import pprint

def sequence_recommendations(book, recommendations):
    expected_order = [
        "Catch 22",
        "The Witching Hour (Lives of the Mayfair Witches)",
        "Interview with the Vampire",
        "The Tale of the Body Thief (Vampire Chronicles (Paperback))",
        "The Vampire Lestat (Vampire Chronicles, Book II)"
    ]
    rec_dict = {title: dist for title, dist in recommendations}
    sequenced = [[title, rec_dict[title]] for title in expected_order if title in rec_dict]
    return [book, sequenced]


books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
books_sequenced = sequence_recommendations(books[0], books[1])
pprint.pprint(books_sequenced)


def test_book_recommendation():
    test_pass = True
    recommends = sequence_recommendations(
        "The Queen of the Damned (Vampire Chronicles (Paperback))",
        get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")[1]
    )

    if recommends[0] != "The Queen of the Damned (Vampire Chronicles (Paperback))":
        test_pass = False

    recommended_books = [
        "Catch 22",
        "The Witching Hour (Lives of the Mayfair Witches)",
        "Interview with the Vampire",
        "The Tale of the Body Thief (Vampire Chronicles (Paperback))",
        "The Vampire Lestat (Vampire Chronicles, Book II)"
    ]

    recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
    
    # Check first two recommendations strictly
    for i in range(2):  
        if recommends[1][i][0] not in recommended_books:
            test_pass = False
        if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
            test_pass = False
            
    if test_pass:
        print("You passed the challenge! 🎉🎉🎉🎉🎉")
    else:
        print("You haven't passed yet. Keep trying!")


test_book_recommendation()